In [1]:
import xml.etree.ElementTree as ET
from scipy.stats import norm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from IPython.display import clear_output

import stellargraph as sg
from stellargraph import StellarGraph
from stellargraph.data import EdgeSplitter
from stellargraph.mapper import FullBatchLinkGenerator
from stellargraph.layer import GCN, LinkEmbedding
from stellargraph import datasets


from tensorflow import keras
from sklearn import preprocessing, feature_extraction, model_selection

from stellargraph import globalvar
from stellargraph import datasets

In [2]:
def analyze_general_metrics(byte_array, row_data):
    unigram_dist_int_chars = [0] * 256
    hamming_ones = 0
    hamming_total_bits = 0
    mean_byte_sum_byte_values = 0
    longest_streak = 0
    entropy_byte_distribution = np.zeros((256,), dtype=int)
    for byte in byte_array:
        ### unigram dist ###
        unigram_dist_int_chars[int(byte.hex(), 16)] += 1
        
        ### hamming weight ###
        bits = bin(int(byte.hex(), 16))[2:]
        hamming_total_bits += len(bits)
        for char in bits:
            if char == '1':
                hamming_ones += 1
                
        ### mean byte value ###
        # convert byte to hex then into int
        mean_byte_sum_byte_values += int(byte.hex(), 16)
                
        ### Longest Streak ###
        i = byte_array.index(byte)
        j = 0
        temp_streak = 1
        while i + j + 1 < len(byte_array) and byte_array[i + j] == byte_array[i + j + 1]:
            temp_streak += 1
            j += 1
        if temp_streak > longest_streak:
            longest_streak = temp_streak
    
    row_data["unigram_dist_mean"], row_data["unigram_dist_std"]  = norm.fit(unigram_dist_int_chars)
    row_data["hamming_weight"] = hamming_ones/hamming_total_bits
    row_data["mean_byte_value"] = mean_byte_sum_byte_values/len(byte_array)
    row_data["longest_streak"] = longest_streak
    row_data["file_size"] = len(byte_array)
    return row_data

In [3]:
def analyzeChilds(root, metrics):
    if len(root) == 0:
        metrics["avg_num_of_attributes"] += len(root.attrib)
        if root.text is not None:
            metrics["num_of_data_chars"] += len(root.text)
            
            for char in root.text:
                if ord(char) <= 128:
                    metrics["data_char_dist"][ord(char)] += 1
            try:
                metrics["nums_a"].append(int(root.text))
            except:
                pass
        if not root.attrib:
            metrics["simple_types"] += 1
        else:
            metrics["complex_types"] += 1
    else:
        metrics["complex_types"] += 1
        metrics["avg_num_of_attributes"] += len(root.attrib)
    for child in root:
            metrics = analyzeChilds(child, metrics)
    return metrics

In [4]:
def analyzeRootChild(root, metrics, root_el, tag):
    if root_el:
        for el_index in range(len(root)):
            
            tag = root[el_index].tag
            
            metrics[tag + "_avg_num_of_attributes"] = 0
            metrics[tag + "_num_of_data_chars"] = 0
            metrics[tag + "_data_char_dist"] = [0] * 128
            metrics[tag + "_nums_a"] = []
            metrics[tag + "_simple_types"] = 0
            metrics[tag + "_complex_types"] = 0
            
            metrics = analyzeRootChild(root, metrics, False, tag)
            
            metrics[tag + "_num_mean"] = norm.fit(np.array(metrics[tag + "_nums_a"], dtype=float))[0]
            metrics[tag + "_num_std"] = norm.fit(np.array(metrics[tag + "_nums_a"], dtype=float))[1]
            metrics[tag + "_data_char_dist_mean"] = norm.fit(metrics[tag + "_data_char_dist"])[0]
            metrics[tag + "_data_char_dist_std"] = norm.fit(metrics[tag + "_data_char_dist"])[1]
            metrics[tag + "_avg_num_of_attributes"] = metrics[tag + "_avg_num_of_attributes"] / metrics[tag + "_complex_types"]
            metrics.pop(tag + "_data_char_dist", None)
            metrics.pop(tag + "_nums_a", None)
    else:
        if len(root) == 0:
            metrics[tag + "_avg_num_of_attributes"] += len(root.attrib)
            if root.text is not None:
                metrics[tag + "_num_of_data_chars"] += len(root.text)
                for char in root.text:
                    if ord(char) <= 128:
                        metrics[tag + "_data_char_dist"][ord(char)] += 1
                try:
                    metrics[tag + "_nums_a"].append(int(root.text))
                except:
                    pass
            if not root.attrib:
                metrics[tag + "_simple_types"] += 1
            else:
                metrics[tag + "_complex_types"] += 1
        else:
            metrics[tag + "_complex_types"] += 1
            metrics[tag + "_avg_num_of_attributes"] += len(root.attrib)
        for child in root:
                metrics = analyzeRootChild(child, metrics, False, tag)
    return metrics

In [5]:
def train_gcn(sdg, train_size, test_size, epochs_input, layer_size_num):
    
    edge_splitter_test = EdgeSplitter(sdg)

    G_test, edge_ids_test, edge_labels_test = edge_splitter_test.train_test_split(
        p=test_size, method="global", keep_connected=False
    )
    
    edge_splitter_train = EdgeSplitter(G_test)

    G_train, edge_ids_train, edge_labels_train = edge_splitter_train.train_test_split(
        p=train_size, method="global", keep_connected=False
    )
    
    epochs = epochs_input
    
    train_gen = FullBatchLinkGenerator(G_train, method="gcn")
    train_flow = train_gen.flow(edge_ids_train, edge_labels_train)
    
    test_gen = FullBatchLinkGenerator(G_test, method="gcn")
    test_flow = train_gen.flow(edge_ids_test, edge_labels_test)
    
    gcn = GCN(
        layer_sizes=[layer_size_num, layer_size_num], activations=["relu", "relu"], generator=train_gen, dropout=0.3
    )
    
    x_inp, x_out = gcn.in_out_tensors()
    
    prediction = LinkEmbedding(activation="relu", method="ip")(x_out)
    
    prediction = keras.layers.Reshape((-1,))(prediction)
    
    model = keras.Model(inputs=x_inp, outputs=prediction)

    model.compile(
        optimizer=keras.optimizers.Adam(lr=0.01),
        loss=keras.losses.binary_crossentropy,
        metrics=["binary_accuracy"],
    )
    
    model = keras.Model(inputs=x_inp, outputs=prediction)

    model.compile(
        optimizer=keras.optimizers.Adam(lr=0.01),
        loss=keras.losses.binary_crossentropy,
        metrics=["binary_accuracy"],
    )
    
    init_train_metrics = model.evaluate(train_flow)
    init_test_metrics = model.evaluate(test_flow)
        
    history = model.fit(
        train_flow, epochs=epochs, validation_data=test_flow, verbose=2, shuffle=False
    )
    
    train_metrics = model.evaluate(train_flow)
    test_metrics = model.evaluate(test_flow)
    
    return train_metrics[1], train_metrics[0], test_metrics[1], test_metrics[0]

In [6]:
def analyzeXML(file, metrics):
    tree = ET.parse(file)
    root = tree.getroot()
    metrics = analyzeRootChild(root, metrics, True, "")
    metrics = analyzeChilds(root, metrics)
    metrics["num_mean"] = norm.fit(np.array(metrics["nums_a"], dtype=float))[0]
    metrics["num_std"] = norm.fit(np.array(metrics["nums_a"], dtype=float))[1]
    metrics["data_char_dist_mean"] = norm.fit(metrics["data_char_dist"])[0]
    metrics["data_char_dist_std"] = norm.fit(metrics["data_char_dist"])[1]
    metrics.pop("data_char_dist", None)
    metrics.pop("nums_a", None)
    metrics["avg_num_of_attributes"] = metrics["avg_num_of_attributes"] / metrics["complex_types"]
    return metrics

In [7]:
num_of_files = 100
num_of_versions = 4
data = []
mapped_edges = []

for xml_num in range(num_of_files):
    
    xml_set = []
    for version in range(num_of_versions):
        xml_path = 'demo_xml/xml_f' + str(xml_num) + '_v' + str(version) + '.xml'
        xml_set.append(xml_path)
    
    for xml_file in xml_set:
        
        xml_name = "xml_" + str(xml_num) + "_v" + str(xml_set.index(xml_file))
        if xml_set.index(xml_file) < len(xml_set) - 1:
            xml_name_next = "xml_" + str(xml_num) + "_v" + str(xml_set.index(xml_file) + 1)
            mapped_edges.append([xml_name, xml_name_next])
        
        metrics = {
            "xml_name": xml_name,
            "simple_types": 0,
            "complex_types": 0,
            "avg_num_of_attributes": 0,
            "num_of_data_chars": 0,
            "data_char_dist": [0] * 128,
            "nums_a": [],
            "data_char_dist_mean": 0,
            "data_char_dist_std": 0,
            "num_mean": 0,
            "num_std": 0
        }
        
        byte_array = []
        with open(xml_path, "rb") as f:
            byte = f.read(1)
            while byte:
                byte_array.append(byte)
                byte = f.read(1)
        metrics = analyze_general_metrics(byte_array, metrics)
        f.close()
        
        metrics = analyzeXML(xml_file, metrics)
        metrics.pop("data_char_dist", None)
        data.append(metrics)
        
        clear_output(wait=True)
        print("Progress: " + str(((num_of_versions * xml_num + xml_set.index(xml_file)) / (num_of_files * num_of_versions)) * 100) + '%')

clear_output(wait=True)
print("Progress: 100%")
node_data = pd.DataFrame(data)
node_data.set_index("xml_name", inplace=True)
node_data.index.name = None
node_data.fillna(0, inplace=True)
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 20)

for column in node_data.columns:
    node_data[column] = node_data[column] /node_data[column].abs().max()
node_data.fillna(0, inplace=True)
node_data

Progress: 100%


,simple_types,complex_types,avg_num_of_attributes,num_of_data_chars,data_char_dist_mean,data_char_dist_std,num_mean,num_std,unigram_dist_mean,unigram_dist_std,...,leo_data_char_dist_mean,leo_data_char_dist_std,dignissim_avg_num_of_attributes,dignissim_num_of_data_chars,dignissim_simple_types,dignissim_complex_types,dignissim_num_mean,dignissim_num_std,dignissim_data_char_dist_mean,dignissim_data_char_dist_std
xml_0_v0,0.000000,0.000211,0.282609,0.000012,0.000012,0.000029,-0.013284,0.000000,0.000210,0.000164,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
xml_0_v1,0.000000,0.000317,0.376812,0.000050,0.000050,0.000086,-0.013284,0.000000,0.000210,0.000164,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
xml_0_v2,0.000000,0.000317,0.376812,0.000056,0.000056,0.000079,0.000000,0.000000,0.000210,0.000164,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
xml_0_v3,0.000000,0.000422,0.423913,0.000088,0.000088,0.000113,0.000000,0.000000,0.000210,0.000164,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
xml_1_v0,0.000000,0.000211,0.565217,0.000009,0.000009,0.000026,0.012807,0.000000,0.000143,0.000106,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
xml_98_v3,0.022200,0.003167,0.000000,0.022693,0.022693,0.022759,0.080522,0.793327,0.015473,0.015952,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
xml_99_v0,0.999849,0.143777,0.000000,0.999947,0.999947,0.999959,0.009381,0.806516,0.706276,0.812447,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
xml_99_v1,0.999849,0.143777,0.000000,0.999953,0.999953,0.999974,0.009407,0.806574,0.706276,0.812447,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
xml_99_v2,0.999924,0.143777,0.000000,0.999982,0.999982,0.999998,0.009407,0.806574,0.706276,0.812447,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
edges = pd.DataFrame(mapped_edges, columns=["source", "target"])
edges.head()

,source,target
0,xml_0_v0,xml_0_v1
1,xml_0_v1,xml_0_v2
2,xml_0_v2,xml_0_v3
3,xml_1_v0,xml_1_v1
4,xml_1_v1,xml_1_v2


In [9]:
sdg = StellarGraph({"xml": node_data}, {"next": edges})
print(sdg.info())

StellarGraph: Undirected multigraph
 Nodes: 400, Edges: 300

 Node types:
  xml: [400]
    Features: float32 vector, length 1174
    Edge types: xml-next->xml

 Edge types:
    xml-next->xml: [300]
        Weights: all 1 (default)
        Features: none


In [ ]:
epochs = 50
results = pd.DataFrame([])
result_a = []
for layer_size in range (1, 3):
    for train_size in range(1, 10):
        for test_size in range(1, 10):
            train_acc, train_loss, test_acc, test_loss = train_gcn(sdg, train_size * 0.1, test_size * 0.1, epochs, layer_size * 16)
            result_a.append({
                'config': 'Layer Size: ' + str(layer_size * 16) + ' Train Size: ' + str(round(train_size * 0.1, 1)) + ' Test Size: ' + str(round(test_size * 0.1, 1)),
                'train_acc': train_acc,
                'train_loss': train_loss,
                'test_acc': test_acc,
                'test_loss': test_loss,
                'layer_size': layer_size,
                'train_size': train_size,
                'test_size': test_size
            })
            print('Progress: ' + str(len(result_a)/2) + '%')
results = pd.DataFrame(result_a)

** Sampled 30 positive and 30 negative edges. **
** Sampled 27 positive and 27 negative edges. **
Using GCN (local pooling) filters...
Using GCN (local pooling) filters...
1/1 [==============================] - 0s 39ms/step - loss: 1.6627 - binary_accuracy: 0.5000
Epoch 1/50
1/1 - 1s - loss: 1.7439 - binary_accuracy: 0.5000 - val_loss: 1.1418 - val_binary_accuracy: 0.5667
Epoch 2/50
1/1 - 0s - loss: 0.8407 - binary_accuracy: 0.6111 - val_loss: 0.9615 - val_binary_accuracy: 0.5167
Epoch 3/50
1/1 - 0s - loss: 0.5782 - binary_accuracy: 0.6852 - val_loss: 1.3163 - val_binary_accuracy: 0.4833
Epoch 4/50
1/1 - 0s - loss: 0.6831 - binary_accuracy: 0.7407 - val_loss: 1.2742 - val_binary_accuracy: 0.5000
Epoch 5/50
1/1 - 0s - loss: 0.9375 - binary_accuracy: 0.7778 - val_loss: 1.4916 - val_binary_accuracy: 0.5167
Epoch 6/50
1/1 - 0s - loss: 0.5916 - binary_accuracy: 0.8148 - val_loss: 1.6964 - val_binary_accuracy: 0.5333
Epoch 7/50
1/1 - 0s - loss: 0.4677 - binary_accuracy: 0.9074 - val_loss: 1.

Epoch 17/50
1/1 - 0s - loss: 0.1671 - binary_accuracy: 0.8958 - val_loss: 1.7771 - val_binary_accuracy: 0.5417
Epoch 18/50
1/1 - 0s - loss: 0.1118 - binary_accuracy: 0.9375 - val_loss: 1.7668 - val_binary_accuracy: 0.5583
Epoch 19/50
1/1 - 0s - loss: 0.1679 - binary_accuracy: 0.9167 - val_loss: 1.8234 - val_binary_accuracy: 0.5667
Epoch 20/50
1/1 - 0s - loss: 0.1252 - binary_accuracy: 0.9375 - val_loss: 1.9952 - val_binary_accuracy: 0.5667
Epoch 21/50
1/1 - 0s - loss: 0.0993 - binary_accuracy: 0.9583 - val_loss: 2.1019 - val_binary_accuracy: 0.5750
Epoch 22/50
1/1 - 0s - loss: 0.1022 - binary_accuracy: 0.9375 - val_loss: 2.1154 - val_binary_accuracy: 0.5750
Epoch 23/50
1/1 - 0s - loss: 0.0910 - binary_accuracy: 0.9792 - val_loss: 2.1697 - val_binary_accuracy: 0.5667
Epoch 24/50
1/1 - 0s - loss: 0.1688 - binary_accuracy: 0.9375 - val_loss: 2.2132 - val_binary_accuracy: 0.5667
Epoch 25/50
1/1 - 0s - loss: 0.1106 - binary_accuracy: 0.9375 - val_loss: 2.4912 - val_binary_accuracy: 0.5667
E

Epoch 36/50
1/1 - 0s - loss: 0.0193 - binary_accuracy: 1.0000 - val_loss: 1.7701 - val_binary_accuracy: 0.6222
Epoch 37/50
1/1 - 0s - loss: 0.0853 - binary_accuracy: 0.9524 - val_loss: 1.7675 - val_binary_accuracy: 0.6222
Epoch 38/50
1/1 - 0s - loss: 0.0804 - binary_accuracy: 0.9762 - val_loss: 1.7027 - val_binary_accuracy: 0.6222
Epoch 39/50
1/1 - 0s - loss: 0.0112 - binary_accuracy: 1.0000 - val_loss: 1.7029 - val_binary_accuracy: 0.6222
Epoch 40/50
1/1 - 0s - loss: 0.0923 - binary_accuracy: 0.9762 - val_loss: 1.7070 - val_binary_accuracy: 0.6222
Epoch 41/50
1/1 - 0s - loss: 0.0981 - binary_accuracy: 0.9524 - val_loss: 1.7150 - val_binary_accuracy: 0.6278
Epoch 42/50
1/1 - 0s - loss: 0.0032 - binary_accuracy: 1.0000 - val_loss: 1.7338 - val_binary_accuracy: 0.6333
Epoch 43/50
1/1 - 0s - loss: 0.0045 - binary_accuracy: 1.0000 - val_loss: 1.7903 - val_binary_accuracy: 0.6278
Epoch 44/50
1/1 - 0s - loss: 0.0430 - binary_accuracy: 0.9762 - val_loss: 1.8941 - val_binary_accuracy: 0.6278
E

1/1 [==============================] - 0s 33ms/step - loss: 1.1564 - binary_accuracy: 0.5300
Epoch 1/50
1/1 - 1s - loss: 1.1870 - binary_accuracy: 0.5000 - val_loss: 0.7955 - val_binary_accuracy: 0.6100
Epoch 2/50
1/1 - 0s - loss: 0.5438 - binary_accuracy: 0.7667 - val_loss: 1.0482 - val_binary_accuracy: 0.5800
Epoch 3/50
1/1 - 0s - loss: 0.2999 - binary_accuracy: 0.9000 - val_loss: 1.7501 - val_binary_accuracy: 0.5567
Epoch 4/50
1/1 - 0s - loss: 0.8412 - binary_accuracy: 0.8333 - val_loss: 1.4008 - val_binary_accuracy: 0.5467
Epoch 5/50
1/1 - 0s - loss: 0.2133 - binary_accuracy: 0.8667 - val_loss: 1.3313 - val_binary_accuracy: 0.5500
Epoch 6/50
1/1 - 0s - loss: 0.1583 - binary_accuracy: 0.9333 - val_loss: 1.2358 - val_binary_accuracy: 0.5667
Epoch 7/50
1/1 - 0s - loss: 0.1079 - binary_accuracy: 0.9667 - val_loss: 1.2576 - val_binary_accuracy: 0.5767
Epoch 8/50
1/1 - 0s - loss: 0.1653 - binary_accuracy: 0.9333 - val_loss: 1.2885 - val_binary_accuracy: 0.5633
Epoch 9/50
1/1 - 0s - loss:

1/1 - 0s - loss: 0.0101 - binary_accuracy: 1.0000 - val_loss: 2.0261 - val_binary_accuracy: 0.5083
Epoch 19/50
1/1 - 0s - loss: 0.0432 - binary_accuracy: 1.0000 - val_loss: 1.9733 - val_binary_accuracy: 0.5167
Epoch 20/50
1/1 - 0s - loss: 0.0588 - binary_accuracy: 0.9583 - val_loss: 1.9448 - val_binary_accuracy: 0.5167
Epoch 21/50
1/1 - 0s - loss: 0.0103 - binary_accuracy: 1.0000 - val_loss: 1.9333 - val_binary_accuracy: 0.5194
Epoch 22/50
1/1 - 0s - loss: 0.0384 - binary_accuracy: 0.9583 - val_loss: 1.9360 - val_binary_accuracy: 0.5194
Epoch 23/50
1/1 - 0s - loss: 0.0199 - binary_accuracy: 1.0000 - val_loss: 1.9780 - val_binary_accuracy: 0.5222
Epoch 24/50
1/1 - 0s - loss: 0.0063 - binary_accuracy: 1.0000 - val_loss: 1.9666 - val_binary_accuracy: 0.5194
Epoch 25/50
1/1 - 0s - loss: 0.0154 - binary_accuracy: 1.0000 - val_loss: 2.0174 - val_binary_accuracy: 0.5222
Epoch 26/50
1/1 - 0s - loss: 0.0063 - binary_accuracy: 1.0000 - val_loss: 2.0510 - val_binary_accuracy: 0.5222
Epoch 27/50
1

...

1/1 [==============================] - 1s 518ms/step - loss: 2.5259 - binary_accuracy: 0.5800
Progress: 56.5%
** Sampled 180 positive and 180 negative edges. **
** Sampled 48 positive and 48 negative edges. **
Using GCN (local pooling) filters...
Using GCN (local pooling) filters...
1/1 [==============================] - 0s 323ms/step - loss: 1.2219 - binary_accuracy: 0.5528
Epoch 1/50
1/1 - 4s - loss: 1.5074 - binary_accuracy: 0.5104 - val_loss: 0.9851 - val_binary_accuracy: 0.6083
Epoch 2/50
1/1 - 1s - loss: 1.0097 - binary_accuracy: 0.6250 - val_loss: 1.4417 - val_binary_accuracy: 0.5917
Epoch 3/50
1/1 - 0s - loss: 1.1982 - binary_accuracy: 0.6458 - val_loss: 1.5361 - val_binary_accuracy: 0.5722
Epoch 4/50
1/1 - 1s - loss: 1.4067 - binary_accuracy: 0.6875 - val_loss: 1.3939 - val_binary_accuracy: 0.5528
Epoch 5/50
1/1 - 0s - loss: 0.9715 - binary_accuracy: 0.8021 - val_loss: 1.3161 - val_binary_accuracy: 0.5556
Epoch 6/50
1/1 - 1s - loss: 0.6099 - binary_accuracy: 0.8646 - val_loss:

Epoch 15/50
1/1 - 1s - loss: 0.0852 - binary_accuracy: 0.9861 - val_loss: 1.7058 - val_binary_accuracy: 0.5810
Epoch 16/50
1/1 - 1s - loss: 0.1006 - binary_accuracy: 0.9583 - val_loss: 1.6954 - val_binary_accuracy: 0.5857
Epoch 17/50
1/1 - 2s - loss: 0.1034 - binary_accuracy: 0.9583 - val_loss: 1.7190 - val_binary_accuracy: 0.5833
Epoch 18/50
1/1 - 1s - loss: 0.0586 - binary_accuracy: 1.0000 - val_loss: 1.8046 - val_binary_accuracy: 0.5833
Epoch 19/50
1/1 - 1s - loss: 0.0879 - binary_accuracy: 0.9861 - val_loss: 1.8907 - val_binary_accuracy: 0.5881
Epoch 20/50
1/1 - 1s - loss: 0.0952 - binary_accuracy: 0.9722 - val_loss: 1.9930 - val_binary_accuracy: 0.5952
Epoch 21/50
1/1 - 0s - loss: 0.0654 - binary_accuracy: 0.9861 - val_loss: 2.0809 - val_binary_accuracy: 0.5905
Epoch 22/50
1/1 - 0s - loss: 0.0697 - binary_accuracy: 1.0000 - val_loss: 2.1799 - val_binary_accuracy: 0.5905
Epoch 23/50
1/1 - 1s - loss: 0.0759 - binary_accuracy: 0.9583 - val_loss: 2.2483 - val_binary_accuracy: 0.5952
E

Epoch 34/50
1/1 - 0s - loss: 0.0370 - binary_accuracy: 0.9792 - val_loss: 2.2503 - val_binary_accuracy: 0.5771
Epoch 35/50
1/1 - 0s - loss: 0.0143 - binary_accuracy: 1.0000 - val_loss: 2.2474 - val_binary_accuracy: 0.5750
Epoch 36/50
1/1 - 0s - loss: 0.0143 - binary_accuracy: 1.0000 - val_loss: 2.2446 - val_binary_accuracy: 0.5792
Epoch 37/50
1/1 - 0s - loss: 0.0203 - binary_accuracy: 0.9792 - val_loss: 2.2654 - val_binary_accuracy: 0.5813
Epoch 38/50
1/1 - 1s - loss: 0.0128 - binary_accuracy: 1.0000 - val_loss: 2.2925 - val_binary_accuracy: 0.5792
Epoch 39/50
1/1 - 1s - loss: 0.0166 - binary_accuracy: 1.0000 - val_loss: 2.3077 - val_binary_accuracy: 0.5750
Epoch 40/50
1/1 - 1s - loss: 0.0423 - binary_accuracy: 0.9792 - val_loss: 2.2863 - val_binary_accuracy: 0.5792
Epoch 41/50
1/1 - 1s - loss: 0.0024 - binary_accuracy: 1.0000 - val_loss: 2.2906 - val_binary_accuracy: 0.5813
Epoch 42/50
1/1 - 1s - loss: 0.0585 - binary_accuracy: 0.9583 - val_loss: 2.2821 - val_binary_accuracy: 0.5813
E

1/1 [==============================] - 0s 256ms/step - loss: 3.3957 - binary_accuracy: 0.5574
Progress: 58.5%
** Sampled 30 positive and 30 negative edges. **
** Sampled 135 positive and 135 negative edges. **
Using GCN (local pooling) filters...
Using GCN (local pooling) filters...
1/1 [==============================] - 0s 329ms/step - loss: 0.9711 - binary_accuracy: 0.6000
Epoch 1/50
1/1 - 3s - loss: 1.0590 - binary_accuracy: 0.5407 - val_loss: 2.3564 - val_binary_accuracy: 0.5500
Epoch 2/50
1/1 - 1s - loss: 1.9676 - binary_accuracy: 0.5444 - val_loss: 1.5401 - val_binary_accuracy: 0.5333
Epoch 3/50
1/1 - 2s - loss: 1.0161 - binary_accuracy: 0.6815 - val_loss: 1.5997 - val_binary_accuracy: 0.5333
Epoch 4/50
1/1 - 3s - loss: 0.7270 - binary_accuracy: 0.7370 - val_loss: 1.3770 - val_binary_accuracy: 0.5333
Epoch 5/50
1/1 - 1s - loss: 0.6747 - binary_accuracy: 0.7185 - val_loss: 1.5613 - val_binary_accuracy: 0.5500
Epoch 6/50
1/1 - 1s - loss: 0.7278 - binary_accuracy: 0.7741 - val_loss:

Epoch 16/50
1/1 - 7s - loss: 0.4602 - binary_accuracy: 0.8792 - val_loss: 1.1792 - val_binary_accuracy: 0.6167
Epoch 17/50
1/1 - 5s - loss: 0.3658 - binary_accuracy: 0.8667 - val_loss: 1.2981 - val_binary_accuracy: 0.6250
Epoch 18/50
1/1 - 5s - loss: 0.4305 - binary_accuracy: 0.8917 - val_loss: 1.3051 - val_binary_accuracy: 0.6250
Epoch 19/50
1/1 - 3s - loss: 0.2930 - binary_accuracy: 0.8875 - val_loss: 1.3063 - val_binary_accuracy: 0.6583
Epoch 20/50
1/1 - 5s - loss: 0.2977 - binary_accuracy: 0.8583 - val_loss: 1.2210 - val_binary_accuracy: 0.6500
Epoch 21/50
1/1 - 8s - loss: 0.3233 - binary_accuracy: 0.8875 - val_loss: 1.1989 - val_binary_accuracy: 0.6500


In [ ]:
pd.set_option('display.max_rows', None)
results.sort_values(by=['test_acc'])

In [ ]:
best_ten_results = pd.DataFrame(results.sort_values(by=['test_acc']).iloc[-10:])
best_ten_results
avg_results_a = []
for index, row in best_ten_results.iterrows():
    train_acc_a = []
    train_loss_a = []
    test_acc_a = []
    test_loss_a = []
    for i in range(0, 10):
        train_acc, train_loss, test_acc, test_loss = train_gcn(sdg, row['train_size'] * 0.1, row['test_size'] * 0.1, 50, row['layer_size'] * 16)
        train_acc_a.append(train_acc)
        train_loss_a.append(train_loss)
        test_acc_a.append(test_acc)
        test_loss_a.append(test_loss)
    print(train_acc_a)
    avg_results_a.append({
        'config': 'Layer Size: ' + str(row['layer_size'] * 16) + ' Train Size: ' + str(round(row['train_size'] * 0.1, 1)) + ' Test Size: ' + str(round(row['test_size'] * 0.1, 1)),
        'Layer Size': row['layer_size'] * 16,
        'Train Size': round(row['train_size'] * 0.1, 1),
        'Test Size': round(row['test_size'] * 0.1, 1),
        'train_acc': norm.fit(train_acc_a)[0],
        'train_loss': norm.fit(train_loss_a)[0],
        'test_acc': norm.fit(test_acc_a)[0],
        'test_loss': norm.fit(test_loss_a)[0]
        
    })
avg_results = pd.DataFrame(avg_results_a)

In [ ]:
avg_results.sort_values(by=['test_acc'])

In [ ]:
avg_results.to_excel("output.xlsx")